<a href="https://colab.research.google.com/github/DanielleRomanoff/DS-Unit-4-Sprint-2-NLP/blob/master/Sprint_Challene_Unit_4_Sprint_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 - Working with Text Data

### Use Python string methods remove irregular whitespace from the following string:

In [0]:
# All imports in one place whether I need them or not. 
import pandas as pd
import numpy as np
import re
import requests
import string

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from bs4 import BeautifulSoup
import gensim
from gensim.models.word2vec import Word2Vec

from collections import defaultdict
from collections import Counter

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.probability import FreqDist
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize

In [55]:
# Removing the unnessary white space to make a normal sentence
whitespace_string = '\n\n  This is a    string   that has  \n a lot of  extra \n   whitespace.   '
print(whitespace_string.strip())
print('---------------')
print(' '.join(whitespace_string.split()))

This is a    string   that has  
 a lot of  extra 
   whitespace.
---------------
This is a string that has a lot of extra whitespace.


### Use Regular Expressions to take the dates in the following .txt file and put them into a dataframe with columns for:

[RegEx dates.txt](https://github.com/ryanleeallred/datasets/blob/master/dates.txt)

- Day
- Month
- Year


In [56]:
# Bringing in the text file to use and make a page with the required information
url = ('https://raw.githubusercontent.com/ryanleeallred/datasets/master/dates.txt')
page= requests.get(url)
page = page.text
page

'March 8, 2015\r\nMarch 15, 2015\r\nMarch 22, 2015\r\nMarch 29, 2015\r\nApril 5, 2015\r\nApril 12, 2015\r\nApril 19, 2015\r\nApril 26, 2015\r\nMay 3, 2015\r\nMay 10, 2015\r\nMay 17, 2015\r\nMay 24, 2015\r\nMay 31, 2015\r\nJune 7, 2015\r\nJune 14, 2015\r\nJune 21, 2015\r\nJune 28, 2015\r\nJuly 5, 2015\r\nJuly 12, 2015\r\nJuly 19, 2015'

In [57]:
# Separate the month, day, and year
month = re.findall(r"[a-zA-z]+", page)
year = re.findall(r"\d{4}", page)
day = re.findall(r"\d+",page)
day = day[::2]
# Create the dataframe
date = pd.DataFrame({'Month ':month,  'Day ': day,  'Year ':year})
date.head()

,Day,Month,Year
0,8,March,2015
1,15,March,2015
2,22,March,2015
3,29,March,2015
4,5,April,2015


# Part 2 - Bag of Words 

### Use the twitter sentiment analysis dataset found at this link for the remainder of the Sprint Challenge:

[Twitter Sentiment Analysis Dataset](https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv)

 ### Clean and tokenize the documents ensuring the following properties of the text:

1) Text should be lowercase.

2) Stopwords should be removed.

3) Punctuation should be removed.

4) Tweets should be tokenized at the word level. 

(The above don't necessarily need to be completed in that specific order.)

### Output some cleaned tweets so that we can see that you made all of the above changes.


In [58]:
# Bringing in the data to read
url = 'https://raw.githubusercontent.com/ryanleeallred/datasets/master/twitter_sentiment_binary.csv'
# Read the data
twitters = pd.read_csv(url)
twitters.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7:30 :O
3,0,.. Omgaga. Im sooo im gunna CRy. I'...
4,0,i think mi bf is cheating on me!!! ...


In [59]:
twitters.isnull().sum()

Sentiment        0
SentimentText    0
dtype: int64

In [60]:
# Making sentiment data readable
twitters['SentimentText'] = twitters['SentimentText'].apply(lambda x: re.sub(r'[^\w]', ' ', x))
twitters.head()

,Sentiment,SentimentText
0,0,is so sad for my APL frie...
1,0,I missed the New Moon trail...
2,1,omg its already 7 30 O
3,0,Omgaga Im sooo im gunna CRy I ...
4,0,i think mi bf is cheating on me ...


In [61]:
# Do everything at once. re: from Ryan's notes
table = str.maketrans('', '', string.punctuation)
stop_words = set(stopwords.words('english'))

def nltk_tokenize(input):

    # word
    tokens = word_tokenize(input)
    #print("Tokens:", tokens)
    #lowercase
    lowercase_tokens = [w.lower() for w in tokens]
    #print("Lowercase:", lowercase_tokens)
    # Strip punctuation
    no_punctuation = [x.translate(table) for x in lowercase_tokens]
    # Remove words that aren't alphabetic
    alphabetic = [word for word in no_punctuation if word.isalpha()]
    # Remove stopwords
    words = [w for w in alphabetic if not w in stop_words]
    return words
  

# Create a new column with the tokenized data
twitters['tokenized'] = twitters['SentimentText'].apply(nltk_tokenize)
twitters.head()

,Sentiment,SentimentText,tokenized
0,0,is so sad for my APL frie...,"[sad, apl, friend]"
1,0,I missed the New Moon trail...,"[missed, new, moon, trailer]"
2,1,omg its already 7 30 O,"[omg, already]"
3,0,Omgaga Im sooo im gunna CRy I ...,"[omgaga, im, sooo, im, gunna, cry, dentist, si..."
4,0,i think mi bf is cheating on me ...,"[think, mi, bf, cheating, tt]"


### How should TF-IDF scores be interpreted? How are they calculated?


*Thie higher the weight value, the more rare the term and vice-versa. The TF-IDF tells us how important or not a word is in a document. Search engines use TF-IDF to find appropriate results for a user's serach.*

*TF = (Number of times the term appears in a document) / (Total number of terms in the document)
IDF = log(number of documents/ Number of times the term appears in all the documents) *

#### **TF-IDF = The TF times the IDF**

# Part 3 - Document Classification

1) Use Train_Test_Split to create train and test datasets.

2) Vectorize the tokenized documents using your choice of vectorization method. 

 - Stretch goal: Use both of the methods that we talked about in class.

3) Create a vocabulary using the X_train dataset and transform both your X_train and X_test data using that vocabulary.

4) Use your choice of binary classification algorithm to train and evaluate your model's accuracy. Report both train and test accuracies.

 - Stretch goal: Use an error metric other than accuracy and implement/evaluate multiple classifiers.



In [73]:
# Making vector getting rid of brackets in tokenized
tokenized_vector = []

for i in twitters['tokenized']:
    making_vector = " ".join(i)
    tokenized_vector.append(making_vector)
    
twitters['tokenized_vector'] = tokenized_vector
twitters.head()

,Sentiment,SentimentText,tokenized,tokenized_vector
0,0,is so sad for my APL frie...,"[sad, apl, friend]",sad apl friend
1,0,I missed the New Moon trail...,"[missed, new, moon, trailer]",missed new moon trailer
2,1,omg its already 7 30 O,"[omg, already]",omg already
3,0,Omgaga Im sooo im gunna CRy I ...,"[omgaga, im, sooo, im, gunna, cry, dentist, si...",omgaga im sooo im gunna cry dentist since supo...
4,0,i think mi bf is cheating on me ...,"[think, mi, bf, cheating, tt]",think mi bf cheating tt


In [74]:
# Assigning X and y values
X = twitters.tokenized_vector
y = twitters.Sentiment

# Splitting data into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Confirming correct shapes of data
print("X_train shape is: ", (X_train.shape))
print("X_test shape is: ",(X_test.shape))
print("y_train shape is: ",(y_train.shape))
print("y_trest shape is: ",(y_test.shape))

X_train shape is:  (79991,)
X_test shape is:  (19998,)
y_train shape is:  (79991,)
y_trest shape is:  (19998,)


In [75]:
# Count Vectorization on training and testing data
vectorizer = CountVectorizer(max_features=100, ngram_range=(1,1), stop_words='english')
vectorizer.fit(X_train)
train_word_counts = vectorizer.transform(X_train)
test_word_counts = vectorizer.transform(X_test)

X_train_vectorized = pd.DataFrame(train_word_counts.toarray(), columns=vectorizer.get_feature_names())
X_test_vectorized = pd.DataFrame(test_word_counts.toarray(), columns=vectorizer.get_feature_names())

print(vectorizer.vocabulary_)

{'sleep': 66, 'quot': 60, 'haha': 25, 'love': 44, 'little': 40, 'got': 22, 'dont': 12, 'know': 36, 'lt': 45, 'twitpic': 84, 'wan': 88, 'na': 51, 'hear': 28, 'like': 39, 'week': 92, 'http': 34, 'com': 7, 'time': 78, 'long': 42, 'thing': 75, 'good': 21, 'night': 55, 'way': 91, 'today': 79, 'yes': 99, 'cool': 9, 'day': 10, 'right': 62, 'lol': 41, 'new': 53, 'nice': 54, 'happy': 26, 'sad': 63, 'come': 8, 'yeah': 98, 'think': 76, 'im': 35, 'really': 61, 'want': 89, 'bad': 3, 'miss': 49, 'help': 29, 'amp': 0, 'twitter': 85, 'gon': 20, 'let': 37, 'oh': 56, 'ok': 57, 'thanks': 73, 'bit': 6, 'said': 64, 'work': 96, 'tomorrow': 80, 'hope': 33, 'feel': 13, 'better': 5, 'soon': 67, 'hi': 31, 'make': 46, 'going': 19, 'follow': 14, 'getting': 16, 'need': 52, 'great': 23, 'ya': 97, 'morning': 50, 'people': 58, 'wait': 87, 'thank': 72, 'welcome': 94, 'ur': 86, 'thats': 74, 'life': 38, 'fun': 15, 'sorry': 68, 'look': 43, 'guys': 24, 'girl': 17, 'wish': 95, 'aww': 2, 'tonight': 81, 'tell': 71, 'home': 3

In [40]:
# Logistic Regression model
logistic_reg = LogisticRegression(random_state=42, solver="lbfgs").fit(X_train_vectorized, y_train)

train_predictions = logistic_reg.predict(X_train_vectorized)
test_predictions = logistic_reg.predict(X_test_vectorized)

print('REPORTS: Logistic Regression')
print('---------------------')
print(f'Train Accuracy: {round(accuracy_score(y_train, train_predictions),4)}')
print(f'Test Accuracy: {round(accuracy_score(y_test, test_predictions),4)}')
print('---------------------')
print(f'Train Roc Auc: {round(roc_auc_score(y_train, train_predictions),4)}')
print(f'Test Roc Auc: {round(roc_auc_score(y_test, test_predictions),4)}')

REPORTS: Logistic Regression
---------------------
Train Accuracy: 0.6546
Test Accuracy: 0.6501
---------------------
Train Roc Auc: 0.6244
Test Roc Auc: 0.6209


In [41]:
# Random Forest Classifier
rand_for= RandomForestClassifier(n_estimators=200).fit(X_train_vectorized, y_train)

train_predictions = rand_for.predict(X_train_vectorized)
test_predictions = rand_for.predict(X_test_vectorized)

print('REPORTS: Random Forest Classifier')
print('---------------------')
print(f'Train Accuracy: {round(accuracy_score(y_train, train_predictions),4)}')
print(f'Test Accuracy: {round(accuracy_score(y_test, test_predictions),4)}')
print('---------------------')
print(f'Train Roc Auc: {round(roc_auc_score(y_train, train_predictions),4)}')
print(f'Test Roc Auc: {round(roc_auc_score(y_test, test_predictions),4)}')

REPORTS: Random Forest Classifier
---------------------
Train Accuracy: 0.7433
Test Accuracy: 0.6364
---------------------
Train Roc Auc: 0.7194
Test Roc Auc: 0.61


In [42]:
# Pipeline Created, Tfidf Vectorizer, Multinomial NB, GridSearch CV
stop = stopwords.words('english')
nb_tfidf = make_pipeline(TfidfVectorizer(stop_words=stop),
                        MultinomialNB())
nb_grid_params = [{'tfidfvectorizer__ngram_range' : [(1,1), (1,2), (1,3)],
                  'tfidfvectorizer__max_features' : [50, 100, None]}]

nb_grid = GridSearchCV(nb_tfidf, nb_grid_params, cv=4)
nb_grid.fit(X_train, y_train)

print('REPORTS: Pipeline')
print('---------------------')
print ('Best Parameters:', nb_grid.best_params_)
print('---------------------')
print ('CV Score', nb_grid.best_score_)
print ('Test Score', nb_grid.score(X_test, y_test))

REPORTS: Pipeline
---------------------
Best Parameters: {'tfidfvectorizer__max_features': None, 'tfidfvectorizer__ngram_range': (1, 1)}
---------------------
CV Score 0.7322948831743571
Test Score 0.7361736173617361


# Part 4 -  Word2Vec

1) Fit a Word2Vec model on your cleaned/tokenized twitter dataset. 

2) Display the 10 words that are most similar to the word "twitter"

In [43]:
!pip install -U gensim
import gensim

    100% |████████████████████████████████| 24.2MB 1.3MB/s 
  Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [0]:
import nltk
nltk.download('all')

In [0]:
tokenized = twitters.tokenized

In [81]:
model = Word2Vec(tokenized, min_count=25, window=5, size=250,negative=10)
dir(model)

['__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_adapt_by_suffix',
 '_check_input_data_sanity',
 '_check_training_sanity',
 '_clear_post_train',
 '_do_train_epoch',
 '_do_train_job',
 '_get_job_params',
 '_get_thread_working_mem',
 '_job_producer',
 '_load_specials',
 '_log_epoch_end',
 '_log_epoch_progress',
 '_log_progress',
 '_log_train_end',
 '_minimize_model',
 '_raw_word_count',
 '_save_specials',
 '_set_train_params',
 '_smart_save',
 '_train_epoch',
 '_train_epoch_corpusfile',
 '_update_job_params',
 '_worker_loop',
 '_worker_loop_corpusfile',
 'accuracy',
 'alpha',
 'batch_words',
 'build_vocab',
 'build_vocab_from_freq',
 'ca

In [82]:
print(model)
print(list(model.wv.vocab))
print(len(model.wv.vocab))

Word2Vec(vocab=3078, size=250, alpha=0.025)
['sad', 'friend', 'missed', 'new', 'moon', 'trailer', 'omg', 'already', 'im', 'sooo', 'gunna', 'cry', 'dentist', 'since', 'get', 'put', 'think', 'mi', 'bf', 'tt', 'worry', 'much', 'chillin', 'sunny', 'work', 'tomorrow', 'tv', 'tonight', 'today', 'miss', 'hmmmm', 'wonder', 'number', 'must', 'positive', 'thanks', 'haters', 'face', 'day', 'weekend', 'sucked', 'far', 'jb', 'isnt', 'showing', 'australia', 'ok', 'thats', 'win', 'lt', 'way', 'feel', 'right', 'man', 'completely', 'useless', 'funny', 'twitter', 'http', 'myloc', 'feeling', 'fine', 'gon', 'na', 'go', 'listen', 'celebrate', 'huge', 'roll', 'thunder', 'scary', 'cut', 'growing', 'well', 'year', 'start', 'happy', 'iran', 'one', 'see', 'cause', 'else', 'following', 'pretty', 'awesome', 'level', 'writing', 'massive', 'blog', 'tweet', 'myspace', 'comp', 'shut', 'lost', 'position', 'headed', 'pull', 'golf', 'place', 'something', 'yeah', 'boring', 'whats', 'wrong', 'please', 'tell', 'wish', 'cou

In [83]:
model.wv.most_similar('twitter', topn=10)

[('dm', 0.843769371509552),
 ('facebook', 0.815216064453125),
 ('updates', 0.809754490852356),
 ('info', 0.8017176389694214),
 ('sent', 0.7987399101257324),
 ('email', 0.7963929772377014),
 ('link', 0.7949679493904114),
 ('message', 0.7924069762229919),
 ('comment', 0.7835289835929871),
 ('list', 0.7774643301963806)]

In [85]:
#  Try more words
model.wv.most_similar('omg')

[('xd', 0.8559877872467041),
 ('soo', 0.8425740003585815),
 ('hahaha', 0.8268434405326843),
 ('sooo', 0.8106974959373474),
 ('lmao', 0.8084343671798706),
 ('yea', 0.807371199131012),
 ('dude', 0.8039300441741943),
 ('wat', 0.7861906886100769),
 ('jk', 0.7763631939888),
 ('dat', 0.7757686972618103)]

In [86]:
# Try more words
model.wv.most_similar('dog')

[('cat', 0.9158025979995728),
 ('knee', 0.9088286757469177),
 ('situation', 0.9078789353370667),
 ('crying', 0.8926919102668762),
 ('holy', 0.8798405528068542),
 ('milk', 0.8762333393096924),
 ('accident', 0.8755646347999573),
 ('darn', 0.8735130429267883),
 ('cats', 0.8711155652999878),
 ('felt', 0.8703235387802124)]